In [100]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import re
import json
import pickle
import datetime
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from datetime import date
from datetime import timedelta
from catboost import Pool, CatBoostClassifier


## СПАРСИТЬ ДОСТУПНЫЕ ГОРОДА

In [5]:
iso_dict = {'RU-AMU':'Амурская область', 'RU-ARK':'Архангельская область', 'RU-AST':'Астраханская область', 'RU-BEL':'Белгородская область', 'RU-BRY':'Брянская область', 'RU-VLA':'Владимирская область', 'RU-VGG':'Волгоградская область', 'RU-VLG':'Вологодская область', 'RU-VOR':'Воронежская область', 'RU-IVA':'Ивановская область', 'RU-IRK':'Иркутская область', 'RU-KGD':'Калининградская область', 'RU-KLU':'Калужская область', 'RU-KEM':'Кемеровская область', 'RU-KIR':'Кировская область', 'RU-KOS':'Костромская область', 'RU-KGN':'Курганская область', 'RU-KRS':'Курская область', 'RU-LEN':'Ленинградская область', 'RU-LIP':'Липецкая область', 'RU-MAG':'Магаданская область', 'RU-MOS':'Московская область', 'RU-MUR':'Мурманская область', 'RU-NIZ':'Нижегородская область', 'RU-NGR':'Новгородская область', 'RU-NVS':'Новосибирская область', 'RU-OMS':'Омская область', 'RU-ORE':'Оренбургская область', 'RU-ORL':'Орловская область', 'RU-PNZ':'Пензенская область', 'RU-PSK':'Псковская область', 'RU-ROS':'Ростовская область', 'RU-RYA':'Рязанская область', 'RU-SAM':'Самарская область', 'RU-SAR':'Саратовская область', 'RU-SAK':'Сахалинская область', 'RU-SVE':'Свердловская область', 'RU-SMO':'Смоленская область', 'RU-TAM':'Тамбовская область', 'RU-TVE':'Тверская область', 'RU-TOM':'Томская область', 'RU-TUL':'Тульская область', 'RU-TYU':'Тюменская область', 'RU-ULY':'Ульяновская область', 'RU-CHE':'Челябинская область', 'RU-YAR':'Ярославская область', 'RU-AD':'Адыгея', 'RU-BA':'Башкортостан', 'RU-BU':'Бурятия', 'RU-DA':'Дагестан', 'RU-IN':'Ингушетия', 'RU-KB':'Кабардино-Балкария', 'RU-KL':'Калмыкия', 'RU-KC':'Карачаево-Черкесия', 'RU-KR':'Карелия', 'RU-ME':'Марий Эл', 'RU-MO':'Мордовия', 'RU-AL':'Республика Алтай', 'RU-KO':'Республика Коми', 'RU-SA':'Якутия', 'RU-SE':'Северная Осетия', 'RU-TA':'Татарстан', 'RU-TY':'Тыва', 'RU-UD':'Удмуртия', 'RU-KK':'Хакасия', 'RU-CE':'Чечня', 'RU-CU':'Чувашия', 'RU-ALT':'Алтайский край', 'RU-ZAB':'Забайкальский край', 'RU-KAM':'Камчатский край', 'RU-KDA':'Краснодарский край', 'RU-KYA':'Красноярский край', 'RU-PER':'Пермский край', 'RU-PRI':'Приморский край', 'RU-STA':'Ставропольский край', 'RU-KHA':'Хабаровский край', 'RU-NEN':'Ненецкий автономный округ', 'RU-KHM':'Ханты-Мансийский автономный округ — Югра', 'RU-CHU':'Чукотский автономный округ', 'RU-YAN':'Ямало-Ненецкий автономный округ', 'RU-SPE':'Санкт-Петербург', 'RU-MOW':'Москва', 'RU-YEV':'Еврейская автономная область',
'RU-SEV':'Севастополь', 'RU-CRY':'Крым'}

capitals = {'RU-AD':'Майкоп', 'RU-ALT':'Барнаул', 'RU-AMU':'Благовещенск', 'RU-ARK':'Архангельск', 'RU-AST':'Астрахань', 'RU-BA':'Уфа', 'RU-BEL':'Белгород', 'RU-BRY':'Брянск', 'RU-BU':'Улан-Удэ', 'RU-VLA':'Владимир', 'RU-VGG':'Волгоград', 'RU-VLG':'Вологда', 'RU-VOR':'Воронеж', 'RU-DA':'Махачкала', 'RU-YEV':'Биробиджан', 'RU-ZAB':'Чита', 'RU-IVA':'Иваново', 'RU-IN':'Назрань', 'RU-IRK':'Иркутск', 'RU-KB':'Нальчик', 'RU-KGD':'Калининград', 'RU-KL':'Элиста', 'RU-KLU':'Калуга', 'RU-KAM':'Петропавловск-Камчатский', 'RU-KC':'Черкесск', 'RU-KR':'Петрозаводск', 'RU-KEM':'Кемерово', 'RU-KIR':'Киров', 'RU-KO':'Сыктывкар', 'RU-KOS':'Кострома', 'RU-KDA':'Краснодар', 'RU-KYA':'Красноярск', 'RU-CRY':'Симферополь', 'RU-KGN':'Курган', 'RU-KRS':'Курск', 'RU-LEN':'Гатчина', 'RU-LIP':'Липецк', 'RU-MAG':'Магадан', 'RU-MO':'Саранск', 'RU-MOW':'Москва', 'RU-MOS':'Домодедово', 'RU-MUR':'Мурманск', 'RU-NEN':'Нарьян-Мар', 'RU-NIZ':'Нижний Новгород', 'RU-NGR':'Великий Новгород', 'RU-NVS':'Новосибирск', 'RU-OMS':'Омск', 'RU-ORE':'Оренбург', 'RU-ORL':'Орел', 'RU-PNZ':'Пенза', 'RU-PER':'Пермь', 'RU-PRI':'Владивосток', 'RU-PSK':'Псков', 'RU-AL':'Горно-Алтайск', 'RU-ME':'Йошкар-Ола', 'RU-ROS':'Ростов-На-Дону', 'RU-RYA':'Рязань', 'RU-SAM':'Самара', 'RU-SPE':'Санкт-Петербург', 'RU-SAR':'Саратов', 'RU-SAK':'Южно-Сахалинск', 'RU-SVE':'Екатеринбург', 'RU-SEV':'Севастополь', 'RU-SE':'Владикавказ', 'RU-SMO':'Смоленск', 'RU-STA':'Ставрополь', 'RU-TAM':'Тамбов', 'RU-TA':'Казань', 'RU-TVE':'Тверь', 'RU-TOM':'Томск', 'RU-TUL':'Тула', 'RU-TY':'Кызыл', 'RU-TYU':'Тюмень', 'RU-UD':'Ижевск', 'RU-ULY':'Ульяновск', 'RU-KHA':'Хабаровск', 'RU-KK':'Абакан', 'RU-KHM':'Ханты-Мансийск', 'RU-CHE':'Челябинск', 'RU-CE':'Грозный', 'RU-CU':'Чебоксары', 'RU-CHU':'Анадырь', 'RU-SA':'Якутск', 'RU-YAN':'Салехард', 'RU-YAR':'Ярославль'}


In [ ]:
driver.get('https://nuipogoda.ru/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F')
regs = [x for x in re.findall('<a class="yf" href="(.*?)">(.*?)</a>', driver.page_source) if x[0][-3:]=='.ru']
cities = []
for reg_ref, reg_name in regs[8:]:
    driver.get('https:' + reg_ref)
    cit = [x for x in re.findall('<a class="yf" href="(.*?)">(.*?)</a>', driver.page_source) if x[0][-3:]=='.ru']
    if (len(cit) == 0):
        city_code = reg_ref.split('.')[0][2:]
        cities.append((reg_ref, city_code, reg_name, reg_name))
    for (city_ref, city_name) in cit:
        city_code = city_ref.split('.')[0][2:]
        cities.append((city_ref, city_code, city_name, reg_name))
    print(reg_name)
cities = list(set(cities))


In [ ]:
tmpdf = pd.DataFrame(cities, columns=['ref', 'code', 'city', 'reg'])
tmpdf = tmpdf[tmpdf.city.apply(lambda t: 'район' not in t)]

badass = tmpdf[['reg']].drop_duplicates()
name_fix = {'Ханты-Мансийский автономный округ':'Ханты-Мансийский автономный округ — Югра', 'Чеченская Республика':'Чечня', 'Чувашская Республика':'Чувашия', 'Республика Карелия':'Карелия', 'Республика Башкортостан':'Башкортостан', 'Республика Саха (Якутия)':'Якутия', 'Республика Крым':'Крым', 'Республика Татарстан':'Татарстан', 'Республика Калмыкия':'Калмыкия', 'Карачаево-Черкесская Республика':'Карачаево-Черкесия', 'Республика Дагестан':'Дагестан', 'Удмуртская Республика':'Удмуртия', 'Республика Адыгея':'Адыгея', 'Республика Марий Эл':'Марий Эл', 'Республика Мордовия':'Мордовия', 'Республика Тыва':'Тыва', 'Республика Ингушетия':'Ингушетия', 'Республика Хакасия':'Хакасия', 'Кабардино-Балкарская Республика':'Кабардино-Балкария', 'Республика Бурятия':'Бурятия', 'Республика Северная Осетия':'Северная Осетия'}
badass.loc[:,'iso'] = badass.reg.apply(lambda t: {v: k for k, v in iso_dict.items()}.get(name_fix.get(t, t)))


tmpdf.loc[:,'iso'] = tmpdf.reg.apply(lambda t: dict(zip(badass.reg, badass.iso)).get(t))


In [ ]:
coords = {}
for n, code in enumerate(tmpdf.code):
    if coords.get(code) is None:
        driver.get('https://' + code + '.nuipogoda.ru/31-декабря#2020')
        t = re.findall('var la=(\d+\.\d+);var lg=(\d+\.\d+)', driver.page_source)
        if (len(t) > 0):
            coords[code] = (lat, lon)

In [ ]:
tmpdf.loc[:,'lat'] = tmpdf.code.apply(lambda t: coords.get(t, (None, None))[0])
tmpdf.loc[:,'lon'] = tmpdf.code.apply(lambda t: coords.get(t, (None, None))[1])
tmpdf = tmpdf[tmpdf.lat.notna() & tmpdf.lon.notna()]
tmpdict = dict(zip([(t, capitals.get(t)) for t in capitals.keys()], [1 for t in capitals.keys()]))
tmpdf.loc[:,'capital'] = tmpdf.apply(lambda t: tmpdict.get((t.iso, t.city), 0), axis=1)
#tmpdf.to_csv('weather.csv', index=False)


In [ ]:
tmpdf.to_csv('khm.csv')

## получение погоды в городе/регионе на дату

In [ ]:
tmpdf = pd.read_csv('weather.csv')
def get_weather(reg, date, driver=None):
    try:
        monthmap = {'01':'января', '02':'февраля', '03':'марта', '04':'апреля', '05':'мая', '06':'июня', '07':'июля', '08':'августа', '09':'сентября', '10':'октября', '11':'ноября', '12':'декабря'}
        code = tmpdf[(tmpdf.iso==reg) & (tmpdf.capital==1)].code.iloc[0]
        yyyy = date.split('-')[0]
        mm = date.split('-')[1]
        dd = date.split('-')[-1].lstrip("0")
        url = 'https://' + code + '.nuipogoda.ru/' + dd + '-' + monthmap.get(mm) + '#' + yyyy
#        driver.get(url)
#        res = [json.loads(x[29:-1]) for x in re.findall('mart\.forecastMap\.set\(.*?\)', driver.page_source)]
        reqtext = requests.get(url).text
        res = [json.loads(x[29:-1]) for x in re.findall('mart\.forecastMap\.set\(.*?\)', reqtext)]
        res_final = []
        for yr in res:
            for t in [[str(datetime.datetime.fromtimestamp(x[0]-10800))] + x for x in yr]:
                res_final.append(t)
        return res_final
    except:
        return [[date + ' 00:00:00', None, None, None, None, None, None, None]]


In [1]:
'''
формат данных:
x[0] - дата/время
x[1] - unixtime (timezone???)
x[2] - тепмература *C
x[3] - давление мм рт ст
x[4] - сила ветра м/с
x[5] - направление ветра *
x[6] - ??? константа
x[7] - облачность %
x[8] - осадки мм
'''
print('')

In [ ]:
for i, date in enumerate(daterange):
    for reg in iso_dict.keys():
        output_file = 'weather_full/' + date + '#' + reg + '.pickle'
        wthr = get_weather(reg, date)
        with open(output_file, 'wb') as f:
            pickle.dump(wthr, f)
    print(date)


In [ ]:
df = pd.read_csv('refined_accidents.csv')
df.loc[:,'date'] = df.event_date.apply(lambda t: str(t)[0:10])

dt = lambda t: datetime.date(int(t[0:4]), int(t[5:7]) ,int(t[8:10]))
daterange = [str(t)[0:10] for t in pd.date_range(dt(df.date.min()), dt(df.date.max()), freq='d')]

for i, date in enumerate(daterange):
    for reg in iso_dict.keys():
        output_file = 'weather_full/' + date + '#' + reg + '.pickle'
        wthr = get_weather(reg, date)
        with open(output_file, 'wb') as f:
            pickle.dump(wthr, f)
    print(date)


In [ ]:
daterange = [str(t)[0:10] for t in pd.date_range(dt(df.date.min()), dt(df.date.max()), freq='d')]

for i, date in enumerate(daterange):
    for reg in iso_dict.keys():
        output_file = 'weather_full/' + date + '#' + reg + '.pickle'
        wthr = get_weather(reg, date)
        with open(output_file, 'wb') as f:
            pickle.dump(wthr, f)
    print(date)


In [ ]:
!ls weather_full/ | wc

In [ ]:
!zip -r weather_full.zip weather_full/

In [6]:
def get_hour_weather(data_weather, h):
    h_w = [x for x in data_weather if x[0]==date+' '+h+':00:00'][0]
    ph_w = [x for x in data_weather if (x[0][4:]==date[4:]+' '+h+':00:00') & (x[0][0:10]<date)]

    temp = h_w[2]
    pres = h_w[3]
    wind = h_w[4]
    cloud = h_w[7]
    prec = h_w[8]

    prev_temp = np.median([t[2] for t in ph_w]) - temp
    prev_pres = np.median([t[3] for t in ph_w]) - pres
    prev_wind = np.median([t[4] for t in ph_w]) - wind
    prev_cloud = np.median([t[7] for t in ph_w]) - cloud
    prev_prec = np.median([t[8] for t in ph_w]) - prec

    return [temp, pres, wind, cloud, prec, prev_temp, prev_pres, prev_wind, prev_cloud, prev_prec]

flatten = lambda t: [item for sublist in t for item in sublist]



In [7]:
resdict = {}

for one_file in [p for p in os.listdir('weather_full/') if p[-7:]=='.pickle']:
    with open('weather_full/' + one_file, 'rb') as f:
        date = one_file.split('#')[0]
        reg = one_file.split('#')[1][:-7]
        data_weather = pickle.load(f)
        features = [None for t in np.arange(10*24)]
        try:
            features = flatten([get_hour_weather(data_weather, ('0'+str(x))[-2:]) for x in np.arange(0,24)])
        except:
            a=1
        resdict[(date, reg)] = features


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
cols = flatten([['temp'+t, 'pres'+t, 'wind'+t, 'cloud'+t, 'prec'+t, 'prev_temp'+t, 'prev_pres'+t, 'prev_wind'+t, 'prev_cloud'+t, 'prev_prec'+t] for t in [('0'+str(x))[-2:] for x in np.arange(0,24)]])
weather_data = pd.DataFrame.from_dict(data=resdict, orient='index', columns=cols).reset_index()
weather_data.loc[:,'date'] = weather_data['index'].apply(lambda t: t[0])
weather_data.loc[:,'reg'] = weather_data['index'].apply(lambda t: t[1])
weather_data = weather_data.drop('index', axis='columns')


In [9]:
weather_data.to_feather('weather_data.feather')


In [153]:
df = pd.read_csv('refined_accidents.csv')
reasons = [t for t in np.unique(df.tags) if t not in ['animals', 'earthquake', 'basement', 'fire']]
df.loc[:,'disaster'] = df.tags.apply(lambda t: int(t in reasons))
df = df[df.disaster==1]
df.loc[:,'region'] = df.region.apply(lambda t: 'RU-'+t if t!='CRM' else 'RU-CRY')
df = df.drop(['tags', 'dataset', 'desc'], axis=1).rename({'event_date':'date', 'region':'reg'}, axis=1)


In [374]:
dt = lambda t: datetime.date(int(t[0:4]), int(t[5:7]) ,int(t[8:10]))
daterange = [str(t)[0:10] for t in pd.date_range(dt(df.date.min()), dt(df.date.max()), freq='d')]
pos = dict(zip(zip(df.date.apply(lambda t: t[0:10]), df.reg), np.repeat(True, df.shape[0])))
neg = [i for i in flatten([[(d, r) for r in iso_dict.keys()] for d in daterange]) if pos.get(i, False)==False]


In [375]:
#негативные примеры
np.random.seed=42
#четверть негативных примеров
neg = [x for x in neg if np.random.uniform()<=1.0]
neg = pd.DataFrame(neg, columns=['date', 'reg'])
neg.loc[:,'date'] = neg.date.apply(lambda t: t+' ' + ('0'+str(np.random.randint(24)))[-2:] + ':00:00')
neg.loc[:,'disaster'] = np.repeat(0, neg.shape[0])


In [376]:
#итоговый датасет
totaldf = df[['date', 'reg', 'disaster']].append(neg)
totaldf.loc[:,'hr'] = totaldf.date.apply(lambda t: int(t[11:13]))
totaldf.loc[:,'d'] = totaldf.date.apply(lambda t: t[:10])


In [377]:
prev_day = lambda t: str(date(int(t[0:4]), int(t[5:7]), int(t[8:10])) - datetime.timedelta(days=1))

In [378]:
#фичи предыдущего дня (240)
prev_day_features = totaldf.apply(
    lambda t: \
    resdict.get(
            (prev_day(t.d), t.reg),
            [None for t in np.arange(240)]
        ),
    axis=1
).values


In [379]:
#фичи текущего дня (240)
this_day_features = totaldf.apply(
    lambda t: \
    resdict.get(
        (t.d, t.reg),
        [None for t in np.arange(240)]
    ),
    axis=1
).values


In [380]:
#общие фичи (480)
features = [t[0]+t[1] for t in zip(prev_day_features, this_day_features)]
#нужные фичи с учетом часа (240)
features = [t[10*(i+1):10*(i+1)+240] for t,i in zip(features, totaldf.hr)]


In [381]:
#генерация колонок (240)
cols = [
#текущий день
    'temp', 'press' ,'wind', 'cloud' ,'prec',
# климат
    'tempH', 'pressH' ,'windH', 'cloudH' ,'precH'
]

postfixes = ['_m'+('0'+str(t))[-2:] for t in np.arange(24)]
postfixes.reverse()

cols = flatten([[f+ps for f in cols] for ps in postfixes])
target = 'disaster'


In [382]:
for i, c in enumerate(cols):
    totaldf.loc[:,c] = [t[i] for t in features]

In [383]:
res_l = pd.read_csv('res_l.csv')
totaldf.loc[:,'powerlines'] = totaldf.reg.apply(lambda t: dict(zip(res_l.iso, res_l.length)).get(t))


In [384]:
totaldf = totaldf.reset_index().drop('index', axis=1)
totaldf.shape

(58654, 246)

In [385]:
totaldf.to_feather('totaldf.feather')


In [408]:
X_train, X_test, y_train, y_test = train_test_split(
    totaldf[cols], totaldf[target],
    stratify=totaldf[target],
    test_size=0.25, random_state=42
)

train = Pool(X_train, y_train)
test = Pool(X_test, y_test)

cboost = CatBoostClassifier(
    iterations=10000,
    learning_rate=0.02,
    auto_class_weights='Balanced',
    depth=2,
    l2_leaf_reg=7.5
)



In [409]:
cboost.fit(
    train,
    eval_set=test,
    early_stopping_rounds=500,
    verbose=True
)


0:	learn: 0.6901637	test: 0.6900198	best: 0.6900198 (0)	total: 18.9ms	remaining: 3m 8s
1:	learn: 0.6872499	test: 0.6869644	best: 0.6869644 (1)	total: 36.3ms	remaining: 3m 1s
2:	learn: 0.6840735	test: 0.6835477	best: 0.6835477 (2)	total: 53.4ms	remaining: 2m 57s
3:	learn: 0.6812489	test: 0.6804477	best: 0.6804477 (3)	total: 70.3ms	remaining: 2m 55s
4:	learn: 0.6784375	test: 0.6774663	best: 0.6774663 (4)	total: 88.6ms	remaining: 2m 57s
5:	learn: 0.6758757	test: 0.6746496	best: 0.6746496 (5)	total: 107ms	remaining: 2m 58s
6:	learn: 0.6734631	test: 0.6720549	best: 0.6720549 (6)	total: 128ms	remaining: 3m 2s
7:	learn: 0.6708180	test: 0.6692575	best: 0.6692575 (7)	total: 146ms	remaining: 3m 2s
8:	learn: 0.6686277	test: 0.6668301	best: 0.6668301 (8)	total: 186ms	remaining: 3m 25s
9:	learn: 0.6662940	test: 0.6643743	best: 0.6643743 (9)	total: 215ms	remaining: 3m 34s
10:	learn: 0.6640066	test: 0.6619236	best: 0.6619236 (10)	total: 255ms	remaining: 3m 51s
11:	learn: 0.6618147	test: 0.6595020	bes

102:	learn: 0.5865593	test: 0.5778195	best: 0.5778195 (102)	total: 2.61s	remaining: 4m 11s
103:	learn: 0.5862317	test: 0.5774614	best: 0.5774614 (103)	total: 2.63s	remaining: 4m 10s
104:	learn: 0.5859668	test: 0.5771751	best: 0.5771751 (104)	total: 2.66s	remaining: 4m 10s
105:	learn: 0.5857018	test: 0.5769632	best: 0.5769632 (105)	total: 2.67s	remaining: 4m 9s
106:	learn: 0.5854773	test: 0.5767043	best: 0.5767043 (106)	total: 2.69s	remaining: 4m 8s
107:	learn: 0.5851952	test: 0.5763798	best: 0.5763798 (107)	total: 2.71s	remaining: 4m 8s
108:	learn: 0.5848727	test: 0.5760239	best: 0.5760239 (108)	total: 2.72s	remaining: 4m 7s
109:	learn: 0.5845976	test: 0.5757116	best: 0.5757116 (109)	total: 2.74s	remaining: 4m 6s
110:	learn: 0.5843981	test: 0.5755138	best: 0.5755138 (110)	total: 2.76s	remaining: 4m 5s
111:	learn: 0.5840240	test: 0.5750936	best: 0.5750936 (111)	total: 2.78s	remaining: 4m 5s
112:	learn: 0.5837969	test: 0.5748285	best: 0.5748285 (112)	total: 2.79s	remaining: 4m 4s
113:	le

194:	learn: 0.5703363	test: 0.5611576	best: 0.5611576 (194)	total: 6.85s	remaining: 5m 44s
195:	learn: 0.5702224	test: 0.5610672	best: 0.5610672 (195)	total: 6.89s	remaining: 5m 44s
196:	learn: 0.5701433	test: 0.5609827	best: 0.5609827 (196)	total: 6.91s	remaining: 5m 43s
197:	learn: 0.5700596	test: 0.5609510	best: 0.5609510 (197)	total: 6.93s	remaining: 5m 42s
198:	learn: 0.5699190	test: 0.5608227	best: 0.5608227 (198)	total: 6.95s	remaining: 5m 42s
199:	learn: 0.5697539	test: 0.5606333	best: 0.5606333 (199)	total: 6.96s	remaining: 5m 41s
200:	learn: 0.5696731	test: 0.5605461	best: 0.5605461 (200)	total: 6.98s	remaining: 5m 40s
201:	learn: 0.5695784	test: 0.5604046	best: 0.5604046 (201)	total: 7s	remaining: 5m 39s
202:	learn: 0.5694752	test: 0.5602691	best: 0.5602691 (202)	total: 7.02s	remaining: 5m 38s
203:	learn: 0.5693789	test: 0.5601819	best: 0.5601819 (203)	total: 7.03s	remaining: 5m 37s
204:	learn: 0.5692863	test: 0.5601132	best: 0.5601132 (204)	total: 7.07s	remaining: 5m 37s
20

290:	learn: 0.5627079	test: 0.5542515	best: 0.5542515 (290)	total: 10.1s	remaining: 5m 35s
291:	learn: 0.5626546	test: 0.5542187	best: 0.5542187 (291)	total: 10.1s	remaining: 5m 36s
292:	learn: 0.5625848	test: 0.5541439	best: 0.5541439 (292)	total: 10.2s	remaining: 5m 38s
293:	learn: 0.5625111	test: 0.5540495	best: 0.5540495 (293)	total: 10.3s	remaining: 5m 39s
294:	learn: 0.5624603	test: 0.5539866	best: 0.5539866 (294)	total: 10.3s	remaining: 5m 39s
295:	learn: 0.5624016	test: 0.5539437	best: 0.5539437 (295)	total: 10.3s	remaining: 5m 39s
296:	learn: 0.5623593	test: 0.5538993	best: 0.5538993 (296)	total: 10.4s	remaining: 5m 38s
297:	learn: 0.5623086	test: 0.5538870	best: 0.5538870 (297)	total: 10.4s	remaining: 5m 38s
298:	learn: 0.5622544	test: 0.5538370	best: 0.5538370 (298)	total: 10.4s	remaining: 5m 37s
299:	learn: 0.5621917	test: 0.5537780	best: 0.5537780 (299)	total: 10.4s	remaining: 5m 37s
300:	learn: 0.5619351	test: 0.5535499	best: 0.5535499 (300)	total: 10.5s	remaining: 5m 36s

390:	learn: 0.5559021	test: 0.5482476	best: 0.5482476 (390)	total: 13.2s	remaining: 5m 25s
391:	learn: 0.5558398	test: 0.5481804	best: 0.5481804 (391)	total: 13.3s	remaining: 5m 25s
392:	learn: 0.5558001	test: 0.5481597	best: 0.5481597 (392)	total: 13.3s	remaining: 5m 25s
393:	learn: 0.5557427	test: 0.5481192	best: 0.5481192 (393)	total: 13.3s	remaining: 5m 24s
394:	learn: 0.5556309	test: 0.5480165	best: 0.5480165 (394)	total: 13.3s	remaining: 5m 24s
395:	learn: 0.5555766	test: 0.5479626	best: 0.5479626 (395)	total: 13.4s	remaining: 5m 24s
396:	learn: 0.5555262	test: 0.5479569	best: 0.5479569 (396)	total: 13.4s	remaining: 5m 23s
397:	learn: 0.5554834	test: 0.5479307	best: 0.5479307 (397)	total: 13.4s	remaining: 5m 23s
398:	learn: 0.5554238	test: 0.5478911	best: 0.5478911 (398)	total: 13.4s	remaining: 5m 23s
399:	learn: 0.5553837	test: 0.5478429	best: 0.5478429 (399)	total: 13.5s	remaining: 5m 23s
400:	learn: 0.5553489	test: 0.5478095	best: 0.5478095 (400)	total: 13.5s	remaining: 5m 22s

...

9601:	learn: 0.3838625	test: 0.4787260	best: 0.4786316 (9504)	total: 3m 47s	remaining: 9.44s
9602:	learn: 0.3838543	test: 0.4787290	best: 0.4786316 (9504)	total: 3m 47s	remaining: 9.42s
9603:	learn: 0.3838462	test: 0.4787284	best: 0.4786316 (9504)	total: 3m 47s	remaining: 9.4s
9604:	learn: 0.3838277	test: 0.4787417	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.39s
9605:	learn: 0.3838191	test: 0.4787471	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.38s
9606:	learn: 0.3838091	test: 0.4787304	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.35s
9607:	learn: 0.3837989	test: 0.4787353	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.33s
9608:	learn: 0.3837901	test: 0.4787339	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.31s
9609:	learn: 0.3837804	test: 0.4787256	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.28s
9610:	learn: 0.3837728	test: 0.4787288	best: 0.4786316 (9504)	total: 3m 48s	remaining: 9.26s
9611:	learn: 0.3837641	test: 0.4787264	best: 0.4786316 (9504)	total: 3m

9691:	learn: 0.3829702	test: 0.4787195	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.41s
9692:	learn: 0.3829583	test: 0.4787262	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.38s
9693:	learn: 0.3829433	test: 0.4787266	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.36s
9694:	learn: 0.3829349	test: 0.4787171	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.33s
9695:	learn: 0.3828939	test: 0.4786954	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.31s
9696:	learn: 0.3828864	test: 0.4786953	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.29s
9697:	learn: 0.3828729	test: 0.4787178	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.26s
9698:	learn: 0.3828638	test: 0.4787138	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.24s
9699:	learn: 0.3828547	test: 0.4787176	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.22s
9700:	learn: 0.3828475	test: 0.4787197	best: 0.4786316 (9504)	total: 3m 53s	remaining: 7.19s
9701:	learn: 0.3828332	test: 0.4786897	best: 0.4786316 (9504)	total: 3

9781:	learn: 0.3821157	test: 0.4786688	best: 0.4786156 (9753)	total: 3m 56s	remaining: 5.28s
9782:	learn: 0.3821061	test: 0.4786769	best: 0.4786156 (9753)	total: 3m 56s	remaining: 5.25s
9783:	learn: 0.3821010	test: 0.4786679	best: 0.4786156 (9753)	total: 3m 56s	remaining: 5.23s
9784:	learn: 0.3820916	test: 0.4786720	best: 0.4786156 (9753)	total: 3m 56s	remaining: 5.21s
9785:	learn: 0.3820733	test: 0.4786741	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.18s
9786:	learn: 0.3820652	test: 0.4786761	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.16s
9787:	learn: 0.3820582	test: 0.4786727	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.13s
9788:	learn: 0.3820492	test: 0.4786670	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.11s
9789:	learn: 0.3820387	test: 0.4786719	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.09s
9790:	learn: 0.3820279	test: 0.4786573	best: 0.4786156 (9753)	total: 3m 57s	remaining: 5.06s
9791:	learn: 0.3820145	test: 0.4786684	best: 0.4786156 (9753)	total: 3

9873:	learn: 0.3812745	test: 0.4785841	best: 0.4785539 (9867)	total: 4m	remaining: 3.07s
9874:	learn: 0.3812670	test: 0.4785774	best: 0.4785539 (9867)	total: 4m	remaining: 3.05s
9875:	learn: 0.3812571	test: 0.4785800	best: 0.4785539 (9867)	total: 4m 1s	remaining: 3.03s
9876:	learn: 0.3812507	test: 0.4785789	best: 0.4785539 (9867)	total: 4m 1s	remaining: 3s
9877:	learn: 0.3812459	test: 0.4785804	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.98s
9878:	learn: 0.3812316	test: 0.4785856	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.95s
9879:	learn: 0.3812225	test: 0.4785766	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.93s
9880:	learn: 0.3812171	test: 0.4785780	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.9s
9881:	learn: 0.3812092	test: 0.4785741	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.88s
9882:	learn: 0.3812013	test: 0.4785690	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.86s
9883:	learn: 0.3811928	test: 0.4785684	best: 0.4785539 (9867)	total: 4m 1s	remaining: 2.83

9966:	learn: 0.3804156	test: 0.4783887	best: 0.4783750 (9964)	total: 4m 4s	remaining: 809ms
9967:	learn: 0.3804045	test: 0.4783821	best: 0.4783750 (9964)	total: 4m 4s	remaining: 785ms
9968:	learn: 0.3803979	test: 0.4783757	best: 0.4783750 (9964)	total: 4m 4s	remaining: 760ms
9969:	learn: 0.3803937	test: 0.4783693	best: 0.4783693 (9969)	total: 4m 4s	remaining: 736ms
9970:	learn: 0.3803859	test: 0.4783775	best: 0.4783693 (9969)	total: 4m 4s	remaining: 711ms
9971:	learn: 0.3803762	test: 0.4783787	best: 0.4783693 (9969)	total: 4m 4s	remaining: 686ms
9972:	learn: 0.3803635	test: 0.4783661	best: 0.4783661 (9972)	total: 4m 4s	remaining: 662ms
9973:	learn: 0.3803499	test: 0.4783569	best: 0.4783569 (9973)	total: 4m 4s	remaining: 637ms
9974:	learn: 0.3803412	test: 0.4783429	best: 0.4783429 (9974)	total: 4m 4s	remaining: 613ms
9975:	learn: 0.3803401	test: 0.4783414	best: 0.4783414 (9975)	total: 4m 4s	remaining: 588ms
9976:	learn: 0.3803310	test: 0.4783391	best: 0.4783391 (9976)	total: 4m 4s	remai

In [410]:
p_train = cboost.predict_proba(train)[:,-1]
p_test = cboost.predict_proba(test)[:,-1]


In [411]:
print(roc_auc_score(y_train, p_train))
print(roc_auc_score(y_test, p_test))

print(f1_score(y_train, [int(x>0.5) for x in p_train]))
print(f1_score(y_test, [int(x>0.5) for x in p_test]))

print(precision_score(y_train, [int(x>0.5) for x in p_train]))
print(precision_score(y_test, [int(x>0.5) for x in p_test]))

print(recall_score(y_train, [int(x>0.5) for x in p_train]))
print(recall_score(y_test, [int(x>0.5) for x in p_test]))


0.9262012949564223
0.8585545366795367
0.5026558975478427
0.4398303192676937
0.3557157569515963
0.3141945773524721
0.8564344160674436
0.7328869047619048


In [412]:
cboost.save_model('no_it_minenergo.cbm', format="cbm")